# Churn Prediction


Run the Mlflow UI:

```sh
uv run --with mlflow==3.7.0 mlflow ui --backend-store-uri sqlite:///mlflow.db
```


Install packages


In [1]:
!uv pip install -q \
    requests==2.32.5

Append notebooks directory to sys.path


In [ ]:
import sys

sys.path.append("../../../..")

Import packages


In [ ]:
import requests

Training pipeline

```py title='app/train.py'
--8<-- "docs/notebooks/python/mlops/deployment-fastapi-churn-prediction/app/train.py"
```


Run pipeline


In [5]:
!cd app && \
    uv run train.py

Installed 81 packages in 1.75s                                   
2026/01/15 16:37:40 INFO mlflow.tracking.fluent: Experiment with name 'telco-churn-prediction' does not exist. Creating a new experiment.
2026/01/15 16:37:47 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
🏃 View run redolent-perch-485 at: http://127.0.0.1:5000/#/experiments/2/runs/01211312ee4c4ec58605a24682e3149a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


Application code

```py title='app/main.py'
--8<-- "docs/notebooks/python/mlops/deployment-fastapi-churn-prediction/app/main.py"
```

Dockerfile

```dockerfile title='app/Dockerfile'
--8<-- "docs/notebooks/python/mlops/deployment-fastapi-churn-prediction/app/Dockerfile"
```

Dependencies

```toml title='app/pyproject.toml'
--8<-- "docs/notebooks/python/mlops/deployment-fastapi-churn-prediction/app/pyproject.toml"
```


Build docker image


In [ ]:
!cd app && \
    docker build -t fastapi-predict-churn-mlflow .


[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (1/3)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 368B                                       0.0s
 => [internal] load metadata for ghcr.io/astral-sh/uv:latest               0.2s
 => [internal] load metadata for docker.io/library/python:3.11-slim-bookw  0.1s
[+] Building 0.4s (1/3)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 368B                                       0.0s
 => [internal] load metadata for ghcr.io/astral-sh/uv:latest               0.3s
 => [internal] load metadata for docker.io/library/python:3.11-slim-bookw  0.3s
[+] Building 0.5s (1/3)                                          docker:default
 => [internal] load build definition fr

Run application


In [31]:
!docker run --rm \
  -it \
  -p 9696:9696 \
  -e AWS_ACCESS_KEY_ID=test \
  -e AWS_SECRET_ACCESS_KEY=test \
  -e AWS_DEFAULT_REGION=us-east-1 \
  -e MLFLOW_S3_ENDPOINT_URL=http://localstack:4566 \
  fastapi-predict-churn-mlflow

Check running applications


In [32]:
!docker ps

CONTAINER ID   IMAGE                          COMMAND                  CREATED          STATUS                      PORTS                                                                  NAMES
3d492ec05127   datadev-mlflow                 "mlflow server --hos…"   12 minutes ago   Up 12 minutes               0.0.0.0:5000->5000/tcp, [::]:5000->5000/tcp                            mlflow
7501595e4faa   postgres:18-alpine3.22         "docker-entrypoint.s…"   12 minutes ago   Up 12 minutes               0.0.0.0:5432->5432/tcp, [::]:5432->5432/tcp                            mlflow-postgres
c7a1b105969d   localstack/localstack:latest   "docker-entrypoint.sh"   12 minutes ago   Up 12 minutes (unhealthy)   4510-4559/tcp, 5678/tcp, 0.0.0.0:4566->4566/tcp, [::]:4566->4566/tcp   datadev-localstack-1


Request application


In [ ]:
url = "http://localhost:9696/predict"

customer = {
    "gender": "female",
    "seniorcitizen": 0,
    "partner": "yes",
    "dependents": "no",
    "phoneservice": "no",
    "multiplelines": "no_phone_service",
    "internetservice": "dsl",
    "onlinesecurity": "no",
    "onlinebackup": "yes",
    "deviceprotection": "no",
    "techsupport": "no",
    "streamingtv": "no",
    "streamingmovies": "no",
    "contract": "month-to-month",
    "paperlessbilling": "yes",
    "paymentmethod": "electronic_check",
    "tenure": 1,
    "monthlycharges": 29.85,
    "totalcharges": 29.85,
}

response = requests.post(url, json=customer)

predictions = response.json()

print(predictions)

ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

Stop container


In [30]:
!docker stop 9743d4689490

Error response from daemon: No such container: 9743d4689490


Delete container image


In [71]:
!docker rmi fastapi-predict-churn

Untagged: fastapi-predict-churn:latest
Deleted: sha256:ace52a950aabd9dad138cecebf4413525f1706e5a43d7e38beba066db1e334be
